In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from pandas.plotting import bootstrap_plot
import plotly.express as px

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore")

import catboost
import xgboost
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score,make_scorer
from xgboost import XGBRegressor

In [ ]:
train_data = pd.read_csv('/kaggle/input/playground-series-s4e5/train.csv')
test_data = pd.read_csv('/kaggle/input/playground-series-s4e5/test.csv')
submission_data = pd.read_csv('/kaggle/input/playground-series-s4e5/sample_submission.csv')

In [ ]:
print(train_data.shape)
print(test_data.shape)
print(submission_data.shape)

data = train_data.drop('id',axis=1)

In [ ]:
data.describe().T

In [ ]:
sns.kdeplot(data['FloodProbability'], fill=True,gridsize=100)
plt.title('FloodProbability')
plt.grid()
plt.show()

In [ ]:
features = data.keys()

In [ ]:
fig, axes = plt.subplots(nrows=7, ncols=3, figsize=(15, 25))
axes = axes.flatten()

for i, feature in enumerate(features):
    sns.violinplot(y=data[feature], ax=axes[i])
    axes[i].set_title(feature)
    axes[i].set_xlabel('')
plt.tight_layout()
plt.show()